# Apply MUVINN with verbose and produce movies 

#### Author: 
Bruno De Santi, PhD
#### Affiliation:
Multi-modality Medical Imaging Lab (M3I Lab), University of Twente, Enschede, The Netherlands
#### Date:
20/09/2023
#### Paper/Project Title:
Automated three-dimensional image registration for longitudinal photoacoustic imaging (De Santi et al. 2023, JBO)
#### GitHub:
https://github.com/brunodesanti/muvinn-reg
#### License:
[Specify the license, e.g., MIT, GPL, etc.]

### Import libraries

In [1]:
# (needs to be changed according to your path)
abs_path = r'C:\Users\DeSantiB\surfdrive\PAMMOTH Study\PAMMOTH image registration\MUVINN-reg'
import sys
sys.path.append(abs_path)

from models import models_extended_verbose as models
from utils import processing as proc
from utils import visualizing as vis
from utils import evaluating as eva

import matplotlib.pyplot as plt
import numpy as np
import os 
import torch
import time
import scipy.ndimage as scnd
import SimpleITK as sitk
import glob
import cv2

plot_flag = True # if one wants to plot figures
save_flag = True # if one wants to save figures

### Apply MUVINN-reg and save results

In [2]:
# Data path (needs to be changed according to your path)
data_path = r'C:\Users\DeSantiB\surfdrive\PAMMOTH Study\PAMMOTH image registration\MUVINN-reg\notebooks\data'

# Path to directory where landmarks are stored (needs to be changed according to your path)
landmarks_path = r'C:\Users\DeSantiB\surfdrive\PAMMOTH Study\PAMMOTH image registration\MUVINN-reg\notebooks\mevislab\landmarks'

# Load and pad fixed image
fixed_acq = '01_01'
fixed_path = data_path + os.path.sep + fixed_acq + '.npy'
fixed_data = np.load(fixed_path, allow_pickle=True).item()
fixed_image = proc.pad_rec(fixed_data["rec"], ((3, 3), (3, 3),(3, 3)))

# Pad depth map and cup mask
depth_map = proc.pad_rec(fixed_data["depth_map"], ((3, 3), (3, 3),(3, 3)))
cup_mask = proc.pad_rec(fixed_data["cup_mask"], ((3, 3), (3, 3),(3, 3)))

# Processing fixed image for image visualization
frangi_options_pp = dict()
frangi_options_pp['sigmas'] = (2, 3, 4)
frangi_options_pp['alpha'] = 0.5
frangi_options_pp['beta'] = 0.5
frangi_options_pp['gamma'] = 1
frangi_options_pp['bw_flag'] = True  

aim_options_pp = dict()
aim_options_pp['half_size_win'] = 5
aim_options_pp['min_sd'] = 0.1
aim_options_pp['weights'] = (0.2, 0.8)

_, _ , fixed_image_pp = proc.processing_vis(fixed_image, frangi_options_pp, aim_options_pp, gpu = 'cuda') 
            
# Adaptive thresholding for vascular segmentation of fixed image
ti = 0.008 # threshold at cup surface
tf = 0.003 # threshold at maximum depth
tau = 100 # decay rate
fixed_mask = proc.segment_vessels(fixed_image_pp, depth_map, ti = ti, tf = tf, tau = tau)

# Define sampling mask
sample_mask = np.bitwise_and(scnd.binary_dilation(cup_mask, np.ones((3, 3, 3))), scnd.binary_dilation(fixed_mask, np.ones((1, 1, 1))))

# Convert to CUDA tensor
fixed_image_t = torch.FloatTensor(fixed_image).cuda()

moving_acqs = ('06_01',)
for moving_acq in moving_acqs:
    
    # Load and pad moving image
    moving_path = data_path + os.path.sep + moving_acq + '.npy'
    moving_data = np.load(moving_path, allow_pickle = True).item() #item() to return each item in tuples
    moving_image = proc.pad_rec(moving_data["rec"], ((3, 3), (3, 3),(3, 3)))
    
    # Process moving image
    _, _ , moving_image_pp = proc.processing_vis(moving_image, frangi_options_pp, aim_options_pp, gpu = 'cuda')

    # Adaptive thresholding for vascular segmentation of moving image
    moving_mask = proc.segment_vessels(moving_image_pp, depth_map, ti = ti, tf = tf, tau = tau)
    
    # Convert to CUDA tensor
    moving_image_t = torch.FloatTensor(moving_image).cuda()
    
    # MUVINN-reg settings and parameters
    kwargs = {}
    kwargs['mask'] = sample_mask

    # Number of epochs
    kwargs['epochs'] = 20000
    # Total number of points to be sampled each iteration
    kwargs['batch_size'] = 200 * (5 ** 3)
    
    # Sigma values during optimization
    kwargs['frangi_sigmas'] = np.array([12, 9, 5, 3, 1.5])
    
    # Epochs for each sigma as percentage of total number of epochs
    frangi_interval = np.array([0.2, 0.2, 0.2, 0.2, 0.2]) 
    kwargs['frangi_epochs'] = np.insert(np.cumsum(np.floor(frangi_interval*kwargs['epochs']))[:-1],0,0)

    # Data term loss function
    kwargs['loss_function'] = 'ncc'
    # Width of local patches around sampled points
    kwargs['ncc_widths'] = 2*kwargs['frangi_sigmas']/100

    # Regularization
    kwargs['jacobian_regularization'] = True
    kwargs['alpha_jacobian'] = 1
    kwargs['hyper_regularization'] = False
    kwargs['alpha_hyper'] = 0.25

    # Network architecture
    kwargs['layers'] = [3,300,300,300,300,300,300,3]
    kwargs['omega'] = 30

    # Optimizer settings
    kwargs['optimizer'] = 'adam' 
    kwargs['lr'] = 5e-5
    
    # Decrease learning rate when decreasing sigma
    kwargs['scheduler_ms'] = kwargs['frangi_epochs'] # MultiStepLR scheduler parameters
    kwargs['scheduler_gamma'] = 0.95

    # If true print results during optimization
    kwargs['verbose'] = True 
    # Folder where to save verbose results if verbose is true (needs to be changed according to your path)
    kwargs['save_folder'] = r'C:\Users\DeSantiB\surfdrive\PAMMOTH Study\PAMMOTH image registration\MUVINN-reg\notebooks\evaluation\verbose'
    # Verbose interval in epochs
    kwargs['log_interval'] = int(kwargs['epochs']/200)
    
    # Frangi parameters inside optimization
    frangi_options = dict()
    frangi_options['alpha'] = 0.5
    frangi_options['beta'] = 0.5
    frangi_options['gamma'] = 0.1
    frangi_options['bw_flag'] = True # White voxels are vessels
    kwargs['frangi_options'] = frangi_options
    
    # Adaptive intensity modulation parameters inside optimization
    aim_options = dict()
    aim_options['half_size_win'] = 5
    aim_options['min_sd'] = 0.1

    # Initialize model
    ImpReg = models.ImplicitRegistrator(moving_image_t, fixed_image_t, **kwargs)
    # Run model optimization
    ImpReg.train()

torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  0%|                                                                                        | 0/20000 [00:00<?, ?it/s]

----------  epoch: 1  ----------
Loss: -0.14134565


  0%|▍                                                                              | 99/20000 [01:16<18:38, 17.80it/s]

----------  epoch: 101  ----------
Loss: -0.79369485


  1%|▊                                                                             | 200/20000 [01:36<18:35, 17.74it/s]

...

 99%|███████████████████████████████████████████████████████████████████████████▌| 19899/20000 [56:10<00:05, 18.63it/s]

----------  epoch: 19901  ----------
Loss: -0.52574855


100%|███████████████████████████████████████████████████████████████████████████▉| 19999/20000 [56:24<00:00, 19.00it/s]

----------  epoch: 20000  ----------
Loss: -0.5189421


100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [56:33<00:00,  5.89it/s]


### Load frames, sort them, write and save movie

In [3]:
# Path where frames are stored
frames_path = r'C:\Users\DeSantiB\surfdrive\PAMMOTH Study\PAMMOTH image registration\MUVINN-reg\notebooks\evaluation\verbose' # verbose folderù

# Specify which frames to select, e.g. for overlays specify "overlay"
frames_name = 'overlay'

# Path where to save movie
output_path = r'C:\Users\DeSantiB\surfdrive\PAMMOTH Study\PAMMOTH image registration\MUVINN-reg\notebooks\evaluation\verbose\overlay.avi'

# Extract filenames and load them in order (iteration number)
numbers = []
for fname in glob.glob(r'{}\{}_*.png'.format(frames_path, frames_name)):
    numbers.append(int(fname.split("_")[-1].split(".")[0]))
s = np.array(numbers)
sort_index = np.argsort(s)
images = [img for img in glob.glob(r'{}\{}_*.png'.format(frames_path, frames_name))]
sort_images = [images[index] for index in sort_index.astype(int)]
frame = cv2.imread(os.path.join(frames_path, images[0]))

# Choose output resolution (same as frame as default)
height, width, _ = frame.shape

# Write and save movie
video = cv2.VideoWriter(output_path, 0, 5, (width, height))
for image in sort_images:
    video.write(cv2.imread(os.path.join(frames_path, image)))
    
# Specify which frames to select, e.g. for overlays specify "overlay"
frames_name = 'depth_overlay'

# Path where to save movie
output_path = r'C:\Users\DeSantiB\surfdrive\PAMMOTH Study\PAMMOTH image registration\MUVINN-reg\notebooks\evaluation\verbose\depth_overlay.avi'

# Extract filenames and load them in order (iteration number)
numbers = []
for fname in glob.glob(r'{}\{}_*.png'.format(frames_path, frames_name)):
    numbers.append(int(fname.split("_")[-1].split(".")[0]))
s = np.array(numbers)
sort_index = np.argsort(s)
images = [img for img in glob.glob(r'{}\{}_*.png'.format(frames_path, frames_name))]
sort_images = [images[index] for index in sort_index.astype(int)]
frame = cv2.imread(os.path.join(frames_path, images[0]))

# Choose output resolution (same as frame as default)
height, width, _ = frame.shape

# Write and save movie
video = cv2.VideoWriter(output_path, 0, 5, (width, height))
for image in sort_images:
    video.write(cv2.imread(os.path.join(frames_path, image)))
    
# Specify which frames to select, e.g. for overlays specify "overlay"
frames_name = 'cor_fixed'

# Path where to save movie
output_path = r'C:\Users\DeSantiB\surfdrive\PAMMOTH Study\PAMMOTH image registration\MUVINN-reg\notebooks\evaluation\verbose\sampling.avi'

# Extract filenames and load them in order (iteration number)
numbers = []
for fname in glob.glob(r'{}\{}_*.png'.format(frames_path, frames_name)):
    numbers.append(int(fname.split("_")[-1].split(".")[0]))
s = np.array(numbers)
sort_index = np.argsort(s)
images = [img for img in glob.glob(r'{}\{}_*.png'.format(frames_path, frames_name))]
sort_images = [images[index] for index in sort_index.astype(int)]
frame = cv2.imread(os.path.join(frames_path, images[0]))

# Choose output resolution (same as frame as default)
height, width, _ = frame.shape

# Write and save movie
video = cv2.VideoWriter(output_path, 0, 5, (width, height))
for image in sort_images:
    video.write(cv2.imread(os.path.join(frames_path, image)))